In [120]:
import os
from glob import glob
import pandas as pd
import numpy as np
from PIL import Image
import os.path as op

In [121]:
# You first need to set you directory structure
# and collect the behavioral files for the localizer and the
# study task separately.  Given that each task will be modeled
# separately treat them separately.
sid = ["001"]
proj_dir = "/Users/chloehampson/Desktop/classes/psb6351/"
# work_dir = '/scratch/madlab/Mattfeld_PSB6351/amattfel/'
behav_dir = op.join(proj_dir, f"raw/sub-{sid[0]}/ses-01/behav")

# Get a list of my study task json and nifti converted files
behav_tsv = sorted(glob(behav_dir + "/*.tsv"))
behav_loc_tsv = behav_tsv[0:2]
behav_task_tsv = behav_tsv[2:7]
# use os.path.join or Pathlib to define location of files
# use glob and sort to grab relevant files...I would separately handle localizer and task .tsv files

In [122]:
behav_task_tsv

['/Users/chloehampson/Desktop/classes/psb6351/raw/sub-001/ses-01/behav/sub-001_task-study_run-1_events.tsv',
 '/Users/chloehampson/Desktop/classes/psb6351/raw/sub-001/ses-01/behav/sub-001_task-study_run-2_events.tsv',
 '/Users/chloehampson/Desktop/classes/psb6351/raw/sub-001/ses-01/behav/sub-001_task-study_run-3_events.tsv',
 '/Users/chloehampson/Desktop/classes/psb6351/raw/sub-001/ses-01/behav/sub-001_task-study_run-4_events.tsv']

In [123]:
# In this cell I'm going to first work on the localizer task

# Define variables that can distinguish between runs and have
# onset times and their duration embedded.  I used dictionaries
# This will need to be accomplished for each type of stimulus or
# or regressor you want to isolate onset times for

# Iterate over your text files for the localizer task.
# The variable curr_behav_file will be a string variable with
# the full path to the separate runs of the localizer task. idx is a counter
# used for indexing.
for idx, curr_behav_file in enumerate(behav_loc_tsv):
    # This is where I defined the keys in my dictionaries that were created above
    # and defined the output associated with each key as an empty list
    # example --> name_of_dictionary_defined_above[f'run{SOMETHING}'] = []

    # I'm using the pandas function read_csv to read in the log files
    curr_behav_data = pd.read_csv(curr_behav_file, sep='\t')

    # Given that the localizer task is a block design you don't need
    # the onset time of each stimulus.  You just need the first and then
    # the total duration of all stimuli to create your blocked time
    # to convolve with your hemodynamic response.  I used temporary lists
    # to grab all onset times and then picked the first...you can choose
    # another way.

    # iterating over ????? here...i is counter for indexing
    # What column header would you want to iterate over?  Why?
    tmp_face_onset = []
    tmp_scene_onset = []
    tmp_math_onset = []
    tmp_face_math = []  
    tmp_scene_math = []

    for i, curr_trial_type in enumerate(curr_behav_data['trial_type']):
        if (
            curr_trial_type == "face" and i % 20 == 0
        ):  # Check if the trial type is 'face' and i is a multiple of 20
            tmp_face_onset.append(
                curr_behav_data["onset"][i]
            )  # Append onset time for 'face' every 20 trials
        elif (
            curr_trial_type == "scence" 
        ):  
            if i % 20 == 10: # Check for 'scene' at every 20 trials
                tmp_scene_onset.append(
                    curr_behav_data["onset"][i]
                )  # Append onset time for 'scene' every 20 trials
        elif (
            curr_trial_type == "math"
        ):  # Check for 'math' at every 20 trials
            # Add to face_math or scene_math based on the original index
            if i % 10 == 0:  # Collect math onsets at every 20th index starting from 0
                tmp_math_onset.append(
                    curr_behav_data["onset"][i]
                )  # Append onset time for 'math' at the 0th index and every 20th after

                # Add to face_math or scene_math based on the original index
                if len(tmp_face_math) <= len(tmp_scene_math):
                    tmp_face_math.append(curr_behav_data["onset"][i])  # Even index for face_math
                else:
                    tmp_scene_math.append(curr_behav_data["onset"][i])  # Odd index for scene_math

    # Convert lists to NumPy arrays for calculations
    face_onset_array = np.array(tmp_face_onset)
    face_math_array = np.array(tmp_face_math)
    scene_onset_array = np.array(tmp_scene_onset)  # Add for scene onsets
    scene_math_array = np.array(tmp_scene_math)  # Add for scene math onsets

    # Ensure the lengths are the same for subtraction for face
    min_length_face = min(len(face_onset_array), len(face_math_array))
    face_duration = (
        face_math_array[:min_length_face] - face_onset_array[:min_length_face]
    )

    # Ensure the lengths are the same for subtraction for scene
    min_length_scene = min(len(scene_onset_array), len(scene_math_array))
    scene_duration = (
        scene_math_array[:min_length_scene] - scene_onset_array[:min_length_scene]
    )

    # Combine onsets and durations into the "onset:duration" format for saving
    face_run_data = np.array(
        [f"{face_onset_array[i]}:{face_duration[i]}" for i in range(min_length_face)]
    )
    scene_run_data = np.array(
        [f"{scene_onset_array[i]}:{scene_duration[i]}" for i in range(min_length_scene)]
    )

    # Save to .1D files in behav_dir
    with open(f"{behav_dir}/loc_face_run_{idx + 1}.1D", "w") as face_file:
        face_file.write(", ".join(face_run_data))

    with open(f"{behav_dir}/loc_scene_run_{idx + 1}.1D", "w") as scene_file:
        scene_file.write(", ".join(scene_run_data))

    # Print paths for confirmation
    print("Saved .1D files:")
    print(f"Face Run {idx + 1}: {behav_dir}/loc_face_run_{idx + 1}.1D")
    print(f"Scene Run {idx + 1}: {behav_dir}/loc_scene_run_{idx + 1}.1D")

    # here I am using the first trial type when it becomes math and the
    # face onset list variable is ???? elements long (just exited a face block)
    # to assign the first element of the tmp_face_onset list variable to the
    # dictionary that I created earlier.
    # the format of AFNI stimulus timing files is the following:
    #
    # onset_time:duration (e.g., 6.3:25, 12.7:25, 22.5:25) --> A block started at 6.3, 12.7, and 22.5 seconds
    # into the experiment and each block lasted for 25 seconds in length...make sure these numbers match your
    # experimental design

Saved .1D files:
Face Run 1: /Users/chloehampson/Desktop/classes/psb6351/raw/sub-001/ses-01/behav/loc_face_run_1.1D
Scene Run 1: /Users/chloehampson/Desktop/classes/psb6351/raw/sub-001/ses-01/behav/loc_scene_run_1.1D
Saved .1D files:
Face Run 2: /Users/chloehampson/Desktop/classes/psb6351/raw/sub-001/ses-01/behav/loc_face_run_2.1D
Scene Run 2: /Users/chloehampson/Desktop/classes/psb6351/raw/sub-001/ses-01/behav/loc_scene_run_2.1D


In [124]:
task_remain_events_onset_times = {f"run{run_num}": [] for run_num in range(1, 5)}
task_facefixb4_bl_onset_times = {f"run{run_num}": [] for run_num in range(1, 5)}

# Dictionaries to store onset times based on accuracy
task_remain_events_correct_onset_times = {
    f"run{run_num}": [] for run_num in range(1, 5)
}
task_remain_events_incorrect_onset_times = {
    f"run{run_num}": [] for run_num in range(1, 5)
}

In [125]:
for idx, curr_behav_file in enumerate(behav_task_tsv):
    curr_behav_data = pd.read_csv(curr_behav_file, sep="\t")

    for i, curr_trial_type in enumerate(curr_behav_data["trial_type"]):
        tmp_fix_onset = curr_behav_data["onset"][i]

        # Check for fixed association trials followed by conditional trials
        if "face" in curr_trial_type or "scene" in curr_trial_type:
            if i > 0:
                if (
                    "face" in curr_behav_data["trial_type"][i - 1]
                    or "scene" in curr_behav_data["trial_type"][i - 1]
                ):
                    if curr_behav_data["acc"][i] == 1:
                        task_remain_events_correct_onset_times[f"run{idx+1}"].append(
                            tmp_fix_onset
                        )
                    elif curr_behav_data["acc"][i] == 0:
                        task_remain_events_incorrect_onset_times[f"run{idx+1}"].append(
                            tmp_fix_onset
                        )

        # Fixed association trials preceding correct/incorrect conditional trials
        elif "conditional" in curr_trial_type:
            if i > 0 and "face" in curr_behav_data["trial_type"][i - 1]:
                if curr_behav_data["acc"][i] == 1:
                    task_remain_events_correct_onset_times[f"run{idx+1}"].append(
                        tmp_fix_onset
                    )
                elif curr_behav_data["acc"][i] == 0:
                    task_remain_events_incorrect_onset_times[f"run{idx+1}"].append(
                        tmp_fix_onset
                    )

        # Face/scene trials preceding baseline trials for MVPA
        elif curr_trial_type == "baseline" and (
            "face" in curr_behav_data["trial_type"][i - 1]
            or "scene" in curr_behav_data["trial_type"][i - 1]
        ):
            task_facefixb4_bl_onset_times[f"run{idx+1}"].append(tmp_fix_onset)

        ###almost works but there are some little problems that need to fixed

In [126]:
for curr_run in ["run1", "run2", "run3", "run4"]:
    for onset_dict in [
        task_remain_events_onset_times,
        task_facefixb4_bl_onset_times,
        task_remain_events_correct_onset_times, #doesnt work
        task_remain_events_incorrect_onset_times, #doesnt work
    ]:
        if len(onset_dict[curr_run]) == 0:
            onset_dict[curr_run].append(-1)

In [127]:

# Function to write onset times to files
def save_onsets(onset_dict, condition_name):
    with open(os.path.join(behav_dir, f"{condition_name}_onsets.txt"), "w") as f:
        for curr_run in ["run1", "run2", "run3", "run4"]:
            # Join onset times for each run, separated by commas, and write to file
            f.write(", ".join(map(str, onset_dict[curr_run])) + "\n")


# Save onset times for each condition
save_onsets(task_remain_events_onset_times, "task_remain_events")
save_onsets(task_facefixb4_bl_onset_times, "task_facefixb4_bl")
save_onsets(task_remain_events_correct_onset_times, "task_remain_events_correct") #doesnt work
save_onsets(task_remain_events_incorrect_onset_times, "task_remain_events_incorrect") #doesnt work

In [128]:
# Similar to above I am creating empty dictionary variables
# for each of the events that I am interested in.
# I will then insert run keys to separate the timing files 
# for the events of interest and their specific runs.


# Here I am iterating over the study behavior files.  There should be
# 4 of them.
'''for idx, curr_behav_file in enumerate(behav_task_tsv):
    # I set the run key for each condition of interest
    # Again if you opt not to use the dictionaries as I did
    # you'll need to utilize a different way. HINT you should
    # update the key for as many events of interest you are
    # attempting to isolate
    
    # I read in the current study run behavioral file
    # NOTE: sometimes it is not good practice to use the same
    # variable names across cells. It may execute but erroneously
    curr_behav_data = pd.read_csv(curr_behav_file, sep='\t')
    
    # I iterate now over the contents of the run specific data.
    # Again...you'll want to isolate the column header that lets
    # you assess the event or trial types of interest
    for i, curr_trial_type in enumerate(curr_behav_data['trial_type']):
        # I am evaluating whether or not the current trail type was a 
        # fixed association that had a conditional trial that followed with a face
        # or a scene
        if 'face' in curr_trial_type or 'scene' in curr_trial_type:
            # if it was either of those grab that onset
            tmp_fix_onset = curr_behav_data['onset'][i]
            # if this is not our first trial (i = counter > 0) - remember python is 0-based
            if i > 0:
                # evaluate whether or not the LAST TRIAL (i-1) was a scene or face fix trial
                # grab the current onset time and assign it to the remaining events.
                # In the analysis that I am interested in pursuing I want to separate these
                # trials from trials where the fix face and scence trials are followed either
                # by a conditional trial or by a baseline trial
                
                if 'face' in curr_behav_data['trial_type'][i-1] or 'scene' in curr_behav_data['trial_type'][i-1]:
                    # Check the accuracy for this trial
                    if curr_behav_data['acc'][i] == 1:
                        # Correct trial, add onset to correct dictionary
                        task_remain_events_correct_onset_times[f'run{idx+1}'].append(tmp_fix_onset)
                    elif curr_behav_data['acc'][i] == 0:
                        # Incorrect trial, add onset to incorrect dictionary
                        task_remain_events_incorrect_onset_times[f'run{idx+1}'].append(tmp_fix_onset)
                        
        # Here I am evaluating whether or not the current trial type is a conditional trial 
        # that was responded to correctly
        elif # grab the onset times for fixed association trials before correct and incorrect conditional trials
             # this will require multiple elif statements.  You'll have to assess what the curr_trial_type is
             # and whether or not that trial was correct or incorrect.  You'll then input that 'tmp_fix_onset'
             # variable that you held onto when you evaluated the above trial types
        # Now I am doing the same thing for trials that precede the perceptual baseline trials
        # These trials will be used for the MVPA anlaysis that is planned.
        elif curr_trial_type == 'baseline' and '?????' in curr_behav_data['trial_type'][i-1]:
                task_facefixb4_bl_onset_times[f'run{idx+1}'].append(tmp_fix_onset)
        
            
# Given that we're setting things up to analyze in AFNI
# you can't have runs that don't have any events in them....or
# you can but you can't have an empty row...thus here I am checking
# to see if the runs are empty and if they are adding a filler (-1)
for curr_run in ['run1', 'run2', 'run3', 'run4']:
    if len(NAME_OF_DICT_FOR_STIMULUS_CONDITION_OF_INTEREST[curr_run]) == 0:
        NAME_OF_DICT_FOR_STIMULUS_CONDITION_OF_INTEREST[curr_run].append(-1)'''
    # the code above would have to be repeated for each variable that is 
    # capturing a stimulus condition of interest. THis is not the most
    # effecient way of coding and can be improved upon.

# Reformatting as before to save as a text file with no square brackets
# use what you did for the localizer task here. If you use the same way as before
# or that I used you'll have to have a separate line for each run of each condition
# This is not a great way to code and could be improved upon

# Creating and checking to see if the directory exists for where you want to save your text files
# use the way you did this for the localizer task


# creating my separate ev files with runs written to each line
# use the way you did this before. HINT if you keep the same way as before
# and how I did it you'll need a separate block of code for each regressor (aka stimulus condition)
# you are trying to model and a separate line within each block for each run
# In the end it should look something like the following in one file:
#
# 6.3, 30.7, 76.5, 137.3, 225.1
# 6.3, 30.7, 76.5, 137.3, 225.1
# 6.3, 30.7, 76.5, 137.3, 225.1
# 6.3, 30.7, 76.5, 137.3, 225.1
# 
# In the example above this stimulus had the exact same onset times across each runs
# your output should in format look like this but your onset times should not be the
# same across runs!!!!



"for idx, curr_behav_file in enumerate(behav_task_tsv):\n    # I set the run key for each condition of interest\n    # Again if you opt not to use the dictionaries as I did\n    # you'll need to utilize a different way. HINT you should\n    # update the key for as many events of interest you are\n    # attempting to isolate\n    \n    # I read in the current study run behavioral file\n    # NOTE: sometimes it is not good practice to use the same\n    # variable names across cells. It may execute but erroneously\n    curr_behav_data = pd.read_csv(curr_behav_file, sep='\t')\n    \n    # I iterate now over the contents of the run specific data.\n    # Again...you'll want to isolate the column header that lets\n    # you assess the event or trial types of interest\n    for i, curr_trial_type in enumerate(curr_behav_data['trial_type']):\n        # I am evaluating whether or not the current trail type was a \n        # fixed association that had a conditional trial that followed with a face\

In [130]:
# This cell is required to run the following 3dDeconvolve calls in the subsequent cells
# Here I am collecting the motion related regressor files that were created for each run
# separately, concatenating them and saving the output.

# the below directory structure will change to match how your data are organized
motion_dir = op.join(proj_dir, 'derivatives/preproc/sub-001/motion1stAFNI')
os.makedirs(motion_dir, exist_ok=True)


In [ ]:
loc_motion_files = sorted(glob(proj_dir + motion_dir + "/st*/*loc*bold.1D"))
loc_mot_dict = {}
for mot_i, curr_loc_motion_file in enumerate(loc_motion_files):
    loc_mot_dict[f"run{mot_i + 1}"] = np.genfromtxt(curr_loc_motion_file)

allruns_loc_motion_data = np.concatenate((loc_mot_dict["run1"], loc_mot_dict["run2"]))
np.savetxt(proj_dir + motion_dir + "/allruns_loc_mot_data.1D", allruns_loc_motion_data)

task_motion_files = sorted(glob(proj_dir + motion_dir + "/st*/*study*bold.1D"))
task_mot_dict = {}
for mot_i, curr_task_motion_file in enumerate(task_motion_files):
    task_mot_dict[f"run{mot_i + 1}"] = np.genfromtxt(curr_task_motion_file)

allruns_task_motion_data = np.concatenate(
    (
        task_mot_dict["run1"],
        task_mot_dict["run2"],
        task_mot_dict["run3"],
        task_mot_dict["run4"],
    )
)
np.savetxt(
    proj_dir + motion_dir + "/allruns_task_mot_data.1D", allruns_task_motion_data
)

In [ ]:
%%bash
# Given that we are running this with no data (see -nodata flag) we can run a quick bash
# command in the cell by using line above.  The matrices that are created and image
# will be created in the directory where this jupyter notebook is running. In my case.
# /home/data/madlab/Mattfeld_PSB6351/mattfeld_2020/code

3dDeconvolve -nodata 608 1.76 \
-concat '1D: 0 304' \
-ortvec /home/amattfel/Mattfeld_PSB6351/derivatives/preproc/sub-021/motion1stAFNI/allruns_loc_mot_data.1D motion \
-polort A \
-local_times \
-num_stimts 2 \
-stim_times_AM1 1 /home/amattfel/Mattfeld_PSB6351/derivatives/first_lvl/sub-021/evs/loc_face_evs.1D "dmBLOCK(1)" -stim_label 1 faces \
-stim_times_AM1 2 /home/amattfel/Mattfeld_PSB6351/derivatives/first_lvl/sub-021/evs/loc_scene_evs.1D "dmBLOCK(1)" -stim_label 2 scenes \
-x1D X.loc.xmat.1D -xjpeg X.loc.jpg


In [ ]:
%%bash

3dDeconvolve -nodata 1420 1.76 \
-concat '1D: 0 355 710 1065' \
-ortvec /home/amattfel/Mattfeld_PSB6351/derivatives/preproc/sub-021/motion1stAFNI/allruns_task_mot_data.1D motion \
-polort A \
-local_times \
-num_stimts 5 \
-stim_times 1 /home/amattfel/Mattfeld_PSB6351/derivatives/first_lvl/sub-021/evs/fix_b4_c_cond_evs.1D "TWOGAMpw(4,5,0.2,12,7)" -stim_label 1 fx_b4_c_cond \
-stim_times 2 /home/amattfel/Mattfeld_PSB6351/derivatives/first_lvl/sub-021/evs/fix_b4_ic_cond_evs.1D "TWOGAMpw(4,5,0.2,12,7)" -stim_label 2 fx_b4_ic_cond \
-stim_times 3 /home/amattfel/Mattfeld_PSB6351/derivatives/first_lvl/sub-021/evs/events_remain_evs.1D "TWOGAMpw(4,5,0.2,12,7)" -stim_label 3 all_remain \
-stim_times 4 /home/amattfel/Mattfeld_PSB6351/derivatives/first_lvl/sub-021/evs/facefix_b4_bl_evs.1D "TWOGAMpw(4,5,0.2,12,7)" -stim_label 4 facefx_b4_bl \
-stim_times 5 /home/amattfel/Mattfeld_PSB6351/derivatives/first_lvl/sub-021/evs/scenefix_b4_bl_evs.1D "TWOGAMpw(4,5,0.2,12,7)" -stim_label 5 scenefx_b4_bl \
-x1D X.task.xmat.1D -xjpeg X.task.jpg


In [ ]:
im_task = Image.open(os.path.join(os.getcwd(), 'X.task.jpg'))
im_task

In [ ]:
im_loc = Image.open(os.path.join(os.getcwd(), 'X.loc.jpg'))
im_loc